In [1]:
from math import floor
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [2]:
import numpy as np

In [3]:
def general(dataframe, percent):
    
    # SPLIT DATAFRAME INTO FEATURES & LABELS
    features = dataframe.loc[:, dataframe.columns != 'label'].to_numpy()
    labels = dataframe[['label']].to_numpy()
    
    # LENGTH TO SPLIT AT
    limit = floor(len(features) * percent)
    
    return {
        'train': {
            'features': features[:limit],
            'labels': np.ndarray.flatten(labels[:limit])
        },
        'test': {
            'features': features[limit:],
            'labels': np.ndarray.flatten(labels[limit:])
        }
    }

In [16]:
def timeseries(dataset, folds, window=0):
    
    # DECONSTRUCT PARAMS
    features = dataset['features']
    labels = dataset['labels']
    
    # INITIALIZE A X FOLD SPLIT
    tss = TimeSeriesSplit(n_splits=folds)
    data = []
    
    # SPLIT DATASET X TIMES
    for train_index, test_index in tss.split(features):
        
        # SHIFT THE VALIDATION INDEX WHEN A SLIDING WINDOW IS GIVEN
        if window:
            test_index = np.concatenate((train_index[-window:], test_index))
        
        # APPEND FOLD DATA
        data.append({
            'train': {
                'features': features[train_index],
                'labels': labels[train_index]
            },
            'test': {
                'features': features[test_index],
                'labels': labels[test_index]
            }
        })
    
    return data

In [9]:
def generator(features, labels, params, shuffle=True):
    
    # DECONSTRUCT PARAMS
    batch = params['batch']
    window = params['window']
    
    # GENERATE & RETURN
    return TimeseriesGenerator(
        features,
        labels,
        length=window,
        batch_size=batch,
        shuffle=shuffle
    )

In [7]:
def windowed(dataset, window):
    
    # CLONE THE ORIGINAL
    secondary = dataset['test'].copy()

    # ADD T
    secondary['features'] = np.concatenate((
        dataset['train']['features'][-window:],
        secondary['features']
    ))

    secondary['labels'] = np.concatenate((
        dataset['train']['labels'][-window:],
        secondary['labels']
    ))
    
    return secondary